In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../')
print(sys.path)

['/home/hieutt/UniCon/notebooks', '/home/hieutt/miniconda3/envs/torchtf/lib/python39.zip', '/home/hieutt/miniconda3/envs/torchtf/lib/python3.9', '/home/hieutt/miniconda3/envs/torchtf/lib/python3.9/lib-dynload', '', '/home/hieutt/miniconda3/envs/torchtf/lib/python3.9/site-packages', '../']


In [39]:
from networks.resnet_big import ConResNet
from networks.efficient_net import ConEfficientNet
from networks.vision_trans import CEViT
from networks.rec_cnn import RecCNN
from torchsummary import summary
from thop import profile
import torch
import torch.nn as nn 

In [15]:
# model = ConResNet()
conenet_model = ConEfficientNet(embedding_dim=1792, feat_dim=128, head='mlp', pretrained=False)
# conenet_model = conenet_model.to('cuda')

In [49]:
summary(conenet_model, input_size=(3, 64, 64), device='cpu')

TypeError: summary() got an unexpected keyword argument 'depth'

In [36]:
x = torch.rand((1, 3, 64, 64))
macs, params = profile(conenet_model, inputs=(x,))

print(f"#Parameters (M): {params / 1e6:.2f}")
print(f"MACs (M): {macs / 1e8:.2f}")

[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register count_normalization() for <class 'torch.nn.modules.batchnorm.BatchNorm2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.padding.ZeroPad2d'>.
[INFO] Register count_adap_avgpool() for <class 'torch.nn.modules.pooling.AdaptiveAvgPool2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.dropout.Dropout'>.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.container.Sequential'>.
#Parameters (M): 6.78
MACs (M): 0.34


In [44]:
conresnet_model = ConResNet(name='resnet50')

In [45]:
summary(conresnet_model, input_size=(3, 64, 64), device='cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 64, 64]           1,728
       BatchNorm2d-2           [-1, 64, 64, 64]             128
            Conv2d-3           [-1, 64, 64, 64]           4,096
       BatchNorm2d-4           [-1, 64, 64, 64]             128
            Conv2d-5           [-1, 64, 64, 64]          36,864
       BatchNorm2d-6           [-1, 64, 64, 64]             128
            Conv2d-7          [-1, 256, 64, 64]          16,384
       BatchNorm2d-8          [-1, 256, 64, 64]             512
            Conv2d-9          [-1, 256, 64, 64]          16,384
      BatchNorm2d-10          [-1, 256, 64, 64]             512
       Bottleneck-11          [-1, 256, 64, 64]               0
           Conv2d-12           [-1, 64, 64, 64]          16,384
      BatchNorm2d-13           [-1, 64, 64, 64]             128
           Conv2d-14           [-1, 64,

In [37]:
x = torch.rand((1, 3, 64, 64))
macs, params = profile(conresnet_model, inputs=(x,))

print(f"#Parameters (M): {params / 1e6:.2f}")
print(f"MACs (M): {macs / 1e9:.2f}")

[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register count_normalization() for <class 'torch.nn.modules.batchnorm.BatchNorm2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.container.Sequential'>.
[INFO] Register count_adap_avgpool() for <class 'torch.nn.modules.pooling.AdaptiveAvgPool2d'>.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
#Parameters (M): 11.50
MACs (M): 2.23


In [30]:
vit_model = CEViT(emb_size=256, n_classes=10)

In [31]:
x = torch.rand((1, 3, 64, 64))
macs, params = profile(vit_model, inputs=(x,))

print(f"#Parameters (M): {params / 1e6:.2f}")
print(f"MACs (M): {macs / 1e6:.2f}")

[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.container.Sequential'>.
[INFO] Register count_normalization() for <class 'torch.nn.modules.normalization.LayerNorm'>.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.dropout.Dropout'>.
#Parameters (M): 9.68
MACs (M): 164.00


In [46]:
summary(vit_model, input_size=(3, 64, 64), device='cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 256, 4, 4]         196,864
         Rearrange-2              [-1, 16, 256]               0
    PatchEmbedding-3              [-1, 17, 256]               0
         LayerNorm-4              [-1, 17, 256]             512
            Linear-5              [-1, 17, 768]         197,376
           Dropout-6            [-1, 8, 17, 17]               0
            Linear-7              [-1, 17, 256]          65,792
MultiHeadAttention-8              [-1, 17, 256]               0
           Dropout-9              [-1, 17, 256]               0
      ResidualAdd-10              [-1, 17, 256]               0
        LayerNorm-11              [-1, 17, 256]             512
           Linear-12             [-1, 17, 1024]         263,168
             GELU-13             [-1, 17, 1024]               0
          Dropout-14             [-1, 1

In [40]:
rec_cnn_model = RecCNN(num_classes=10)

In [43]:
x = torch.rand((1, 3, 64, 64))
macs, params = profile(rec_cnn_model, inputs=(x,))

print(f"#Parameters (M): {params / 1e6:.2f}")
print(f"MACs (M): {macs / 1e6:.2f}")

[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.pooling.MaxPool2d'>.
[INFO] Register count_adap_avgpool() for <class 'torch.nn.modules.pooling.AdaptiveAvgPool2d'>.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
#Parameters (M): 2.47
MACs (M): 160.20


In [47]:
summary(rec_cnn_model, input_size=(3, 64, 64), device='cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 64, 64]           1,792
         MaxPool2d-2           [-1, 64, 32, 32]               0
            Conv2d-3          [-1, 128, 32, 32]          73,856
         MaxPool2d-4          [-1, 128, 16, 16]               0
            Conv2d-5          [-1, 256, 16, 16]         295,168
         MaxPool2d-6            [-1, 256, 8, 8]               0
 AdaptiveAvgPool2d-7            [-1, 256, 4, 4]               0
            Linear-8                  [-1, 512]       2,097,664
            Linear-9                   [-1, 10]           5,130
Total params: 2,473,610
Trainable params: 2,473,610
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.05
Forward/backward pass size (MB): 4.41
Params size (MB): 9.44
Estimated Total Size (MB): 13.89
--------------------------------------